In [1]:
import csv
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [2]:
urls = [
    "https://www.tripadvisor.com/Hotel_Review-g60763-d8873263-Reviews-Innside_New_York_NoMad-New_York_City_New_York.html",
    "https://www.tripadvisor.com/Hotel_Review-g60763-d23448880-Reviews-Motto_by_Hilton_New_York_City_Chelsea-New_York_City_New_York.html",
    "https://www.tripadvisor.com/Hotel_Review-g60763-d12551350-Reviews-Pod_Times_Square-New_York_City_New_York.html",
    "https://www.tripadvisor.com/Hotel_Review-g60763-d7182804-Reviews-Residence_Inn_New_York_Downtown_Manhattan_world_Trade_Center_Area-New_York_City_New_Yor.html",
    "https://www.tripadvisor.com/Hotel_Review-g60763-d5040757-Reviews-Homewood_Suites_by_Hilton_New_York_Midtown_Manhattan_Times_Square_South_NY-New_York_Cit.html",
    "https://www.tripadvisor.com/Hotel_Review-g60763-d8515751-Reviews-Hotel_Riu_Plaza_New_York_Times_Square-New_York_City_New_York.html"
]

In [6]:
# open the file to save the review
path_to_file = '/Users/ronak/Desktop/HotelReviews.csv'
csvFile = open(path_to_file, 'a', encoding="utf-8")
csvWriter = csv.writer(csvFile)
csvWriter.writerow(['link', 'hotel_name', 'overall_rating',  'date_of_review', 'user_name', 'review_rating', 'review_title', 'review_text', 'date_of_stay', 'contributions', 'helpful_votes', 'value_rating', 'rooms_rating', 'location_rating', 'clean_rating', 'service_rating', 'sleep_rating'])


In [9]:
for link in urls:
    path_to_file = link
    driver = webdriver.Safari()
    driver.get(link)
    time.sleep(2)
    
    header = hotel_name = ''
    hotel = None
    overall_rating = 0
    
    # header of hotel
    try:
        header = driver.find_element(By.XPATH, ".//div[contains(@data-test-target, 'hr-aft-info')]")
    except:
        print('Error: header of hotel')
    
    # hotel name
    try:
        hotel = header.find_elements(By.XPATH, ".//div[contains(@class, 'eIsCM f')]")
        hotel_name = hotel[0].text
    except:
        print('Error: hotel name')
    
    # overall rating
    try:
        overall_rating = header.find_element(By.XPATH, ".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
        overall_rating = int(overall_rating)/10
    except:
        print('Error: overall rating')
    
    pages_to_scrape = 20
    
    for i in range(0, pages_to_scrape):
        print('Page no.: ', i)

        time.sleep(2) 

    #     driver.find_element(By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]").click()

        container = driver.find_elements(By.XPATH, "//div[@data-reviewid]")
        user_header_1 = driver.find_elements(By.CLASS_NAME, 'bcaHz')
        user_header_2 = driver.find_elements(By.CLASS_NAME, 'BZmsN')

        for j in range(len(container)):
            print('Review no.: ', j)

            date_of_review = user = location = title = review = date_of_stay = ''
            read_more = None
            contributions = helpful_votes = rating = 0
            value_rating = rooms_rating = location_rating = clean_rating = service_rating = sleep_rating = 0
            # header of the review

            # review date
            try:
                date_of_review = " ".join(user_header_1[j].text.split(" ")[-2:])
            except:
                print('Error: review date')

            # username
            try:
                user = user_header_1[j].text.split("wrote")[0]
            except:
                print('Error: user')

            # location of user
            try:
                location = user_header_2[j].find_element(By.XPATH, ".//span[contains(@class, 'default ShLyt small')]").text
            except:
                print('Error: location')

            # contributions and votes
            try:
                contr_and_votes = user_header_2[j].find_elements(By.XPATH, ".//span[contains(@class, 'eUTJT')]")
                for k in contr_and_votes:
                    k = k.text.split(' ')
                    if k[1] == 'contributions':
                        contributions = k[0]
                    if k[1] == 'helpful':
                        helpful_votes = k[0]
            except:
                print('Error: contributions and votes')

            # main section

            # click on read more of the reviews
            try:
                read_more = container[j].find_element(By.XPATH, ".//div[contains(@data-test-target, 'expand-review')]")
                #driver.execute_script("arguments[0].click();", read_more)
            except:
                print('Error: read more')

            # ratings
            try:
                rating = container[j].find_element(By.XPATH, ".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
                rating = int(rating)/10
            except:
                print('Error: ratings')

            # title of review
            try:
                title = container[j].find_element(By.XPATH, ".//div[contains(@data-test-target, 'review-title')]").text
            except:
                print('Error: title')

            # review text
            try:
                review = container[j].find_element(By.XPATH, ".//q[@class='XllAv H4 _a']").text.replace("\n", "  ")
            except:
                print('Error: review text')

            # date of stay
            try:
                date_of_stay = container[j].find_element(By.XPATH, ".//span[contains(@class, 'euPKI _R Me S4 H3')]").text
                date_of_stay_child = container[j].find_element(By.XPATH, ".//span[contains(@class, 'CrxzX')]").text
                date_of_stay = date_of_stay.replace(date_of_stay_child, '')
            except:
                print('Error: date of stay')

            # Additional ratings
            try:
                additional_ratings = container[j].find_elements(By.XPATH, ".//div[contains(@class, 'fFwef S2 H2 cUidx')]")
                for k in additional_ratings:
                    x = k.find_element(By.XPATH, ".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]
                    x = int(x)/10
                    if k.text == 'Value':
                        value_rating = x
                    elif k.text == 'Rooms':
                        rooms_rating = x
                    elif k.text == 'Location':
                        location_rating = x
                    elif k.text == 'Cleanliness':
                        clean_rating = x
                    elif k.text == 'Service':
                        service_rating = x
                    elif k.text == 'Sleep Quality':
                        sleep_rating = x
                    else:
                        pass
            except:
                print('Error: additional ratings')

            # write to CSV
            try:
                csvWriter.writerow([link, hotel_name, overall_rating,  date_of_review, user, rating, title, review, date_of_stay, contributions, helpful_votes, value_rating, rooms_rating, location_rating, clean_rating, service_rating, sleep_rating])
            except:
                print('Error: writing data to CSV')

        # Next page
        try:
            driver.find_element(By.XPATH, './/a[@class="ui_button nav next primary "]').click()
        except:
            print('Error: next page')
            break

    driver.quit()

Page no.:  0
Review no.:  0
Error: location
Review no.:  1
Review no.:  2
Review no.:  3
Review no.:  4
Review no.:  5
Error: location
Review no.:  6
Error: location
Review no.:  7
Review no.:  8
Review no.:  9
Error: location
Page no.:  1
Review no.:  0
Error: location
Review no.:  1
Review no.:  2
Review no.:  3
Review no.:  4
Error: location
Review no.:  5
Review no.:  6
Review no.:  7
Error: location
Review no.:  8
Error: location
Review no.:  9
Page no.:  2
Review no.:  0
Review no.:  1
Error: location
Review no.:  2
Review no.:  3
Review no.:  4
Review no.:  5
Review no.:  6
Review no.:  7
Review no.:  8
Error: location
Review no.:  9
Page no.:  3
Review no.:  0
Review no.:  1
Review no.:  2
Error: location
Review no.:  3
Error: location
Review no.:  4
Review no.:  5
Review no.:  6
Error: location
Review no.:  7
Review no.:  8
Error: location
Review no.:  9
Error: location
Page no.:  4
Review no.:  0
Error: location
Review no.:  1
Review no.:  2
Error: location
Review no.:  3
Rev